In [27]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [28]:
frame = 15
width = 256
height = 256

In [29]:
def load_video_names(path):
    videos = []
    labels = []
    for category in os.listdir(path):
        for video in os.listdir(path+"/"+category):
            videos.append(path+"/"+category+"/"+video)
            labels.append(category)
    
    return np.array(videos),np.array(labels)

In [30]:
def preprocess(frame):
    frame = cv2.resize(frame, (width, height))
    frame = frame - 127.5
    frame = frame + 127.5
    return frame

In [31]:
def load_video(video_path):
    video_frames = []
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if ret == True:
            video_frames.append(preprocess(frame))
        else:
            break
    cap.release()
    video_frames = select_frames(video_frames)
    if len(video_frames) != frames:
        print("short_video",video_path, len(video_frames))
        
    return np.array(video_frames)

In [32]:
def select_frames(video_frames):
    selected_frames = []
    if len(video_frames) > frames:
        fn = len(video_frames)//frames
        f_num = 0
        for f in video_frames:
            if len(selected_frames) < frames:
                if f_num % fn == 0:
                    selected_frames.append(f)
            f_num +=1
            
    else:
        selected_frames = video_frames
    
    return selected_frames

In [33]:
#data = os.listdir(r"D:\Video_data\train")
#label_types = os.listdir(r"D:\Video_data\train")

In [34]:
def create_dataset(videos, labels, index):
    x = []
    y = []
    for video, label in zip(videos[index], labels[index]):
        x.append(load_video(video))
        y.append(label)
    
    return np.array(x), np.array(y)
    

In [35]:
videos, labels = load_video_names("D:/Video_data/train")
samples = len(videos)

In [36]:
samples

10

In [37]:
classes = len(np.unique(labels))
classes

3

In [38]:
label_counts = np.unique(labels, return_counts = True)
for l, n in zip(label_counts[0], label_counts[1]):
    print(l, n)

fire 5
nofire 2
smoke 3


In [39]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

le = LabelEncoder()
le.fit(np.unique(labels))
encoded_labels = le.transform(labels)
encoded_labels = np.reshape(encoded_labels, (-1,1))

encoder = OneHotEncoder()
encoder.fit(encoded_labels)
encoded_labels = encoder.transform(encoded_labels)

In [40]:
encoded_labels = encoded_labels.toarray()

In [41]:
import tensorflow as tf
from tensorflow.keras import Input

In [42]:
from keras.models import Model
from keras.layers import TimeDistributed, LSTM
from keras.layers import ConvLSTM2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, LeakyReLU, BatchNormalization
from keras.layers import Dense, Flatten, GlobalMaxPooling2D
from keras.layers import MaxPooling3D
from keras.layers import concatenate
from keras.optimizers import Adam

In [43]:
def res_block(model, filters):
    start_block = model
    model = Conv2D(filters = filters, kernel_size = 3, padding = "same")(model)
    model = BatchNormalization(momentum = 0.9)(model)
    model = LeakyReLU(0.2)(model)
    return concatenate([start_block, model])

In [44]:
def create_model():
    input_layer = Input(shape = (frame, width, height, 3))
    
    model = ConvLSTM2D(32, 3, padding = "same", return_sequences = False)(input_layer)
    model = BatchNormalization(momentum = 0.9)(model)
    model = LeakyReLU(0.2)(model)
    
    filters = 64
    
    for _ in range(6):
        model = res_block(model, filters)
        try:
            model = MaxPooling3D((2,2,2))(model)
        except:
            model = MaxPooling2D((2,2))(model)
            
        if filters < 512:
            filters*=2
    
    model = Flatten()(model)
    model = Dense(classes, activation = "softmax")(model)
    model = Model(input_layer, model)
    
   # leraning_rate = 0.0001
    model.compile(optimizer = Adam(learning_rate = 0.0001),
                 loss = "categorical_crossentropy",
                 metrics = ['accuracy'])
    
    return model
    

In [45]:
classifier = create_model()

In [46]:
classifier.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 15, 256, 25  0           []                               
                                6, 3)]                                                            
                                                                                                  
 conv_lstm2d_1 (ConvLSTM2D)     (None, 256, 256, 32  40448       ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 256, 256, 32  128        ['conv_lstm2d_1[0][0]']          
 rmalization)                   )                                                           

 batch_normalization_13 (BatchN  (None, 8, 8, 512)   2048        ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 leaky_re_lu_13 (LeakyReLU)     (None, 8, 8, 512)    0           ['batch_normalization_13[0][0]'] 
                                                                                                  
 concatenate_11 (Concatenate)   (None, 8, 8, 2016)   0           ['max_pooling2d_10[0][0]',       
                                                                  'leaky_re_lu_13[0][0]']         
                                                                                                  
 max_pooling2d_11 (MaxPooling2D  (None, 4, 4, 2016)  0           ['concatenate_11[0][0]']         
 )                                                                                                
          

In [47]:
import keras.utils.vis_utils
from importlib import reload
reload(keras.utils.vis_utils)

<module 'keras.utils.vis_utils' from 'D:\\Personal\\codes\\python\\lib\\site-packages\\keras\\utils\\vis_utils.py'>

In [49]:
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
plot_model(classifier)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [46]:
model_name = "D:/Video_data_model/conv_lstm_2d.h5"

In [48]:
try:
    classifier.load_weights(model_name)
except:
    pass

In [53]:
batch_size = 4
if samples//batch_size < samples/batch_size:
    batches = (samples//batch_size)+1
else:
    batches = samples//batch_size

    for e in range(100):
        indexi = list(raange(samples))
        np.random.shuffle(indexi)
        accuracy = 0
        for batch in raange(batches):
            bs = batch*baatch_size
            be = bs + batch_size
            seelected_indexes = indexi[bs:be]
            
            x,y = create_dataset(videos, encoded_labels, selected_indexes)
            results = classifier.train_on_batch(x,y)
            print("\r",batch,"/",batches,":",results[0],results[1],end = " ")
            accuracy += results[1]
            if batch%100 == 0:
                classifier.save_weights(model_name)
        
        print("\r",e,",Accuracy=", accuracy/batches)
        classifier.save_weights(model_name)
    